# Importazione librerie e setup

In [1]:
from IPython.core.display import display, HTML

display(HTML("<style>.container { width:90% !important; }</style>"))

In [2]:
import pandas as pd
import numpy as np
import math as m
import seaborn as sns
from pandas.api.types import CategoricalDtype

In [3]:
path = './data/'

# Conversione files e Importazione datasets

## Convesione da `.csv` a `.parquet`

Data la grande dimensione di alcuni dei dataset utilizzati, questi verranno convertiti da `.csv` a `.parquet` (Apache Parquet Format)

In [4]:
#df = pd.read_csv('./Datasets/Dati_sensori_meteo_2021.csv', low_memory=False)
#df.to_parquet('./Datasets/Dati_sensori_meteo_2021.parquet')

## Dataset Meteo - group

### Attributi

- IdSensore: 
    - Tipologia: Testo normale
- Data: 
    - Data e ora, Marcatura oraria flessibile
- Valore: 
    - Tipologia: numero
    - Legenda: 9999 = dato mancante 888, 8888 = direzione vento variabile 777, 7777 = calma (solo per direzione di vento)
- idOperatore	
    - *Molto utile per capire la dimensione del dato*
    - LEGENDA: 1: Valore medio 3: Valore massimo 4: Valore cumulato (per la pioggia)
    - Testo normale
- Stato	
    - LEGENDA: VA, VV = dato valido NA, NV, NC = dato invalido NI = dato incerto ND = dato non disponibile
    - Testo normale

### Read_csv

In [5]:
meteo__id_day = pd.read_csv(f'{path}data_meteo/meteo_group_17_21.csv', parse_dates = ['Data'], index_col = 0)
meteo__id_day.dtypes

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


IdSensore                int64
Data            datetime64[ns]
Valore_mean            float64
Valore_std             float64
Valore_max             float64
Valore_count             int64
dtype: object

In [6]:
meteo__id_day

,IdSensore,Data,Valore_mean,Valore_std,Valore_max,Valore_count
0,3,2017-01-01,37.054861,0.934560,38.3,144
1,3,2017-01-02,37.584028,0.618894,38.3,144
2,3,2017-01-03,37.242361,0.567238,38.2,144
3,3,2017-01-04,37.350694,0.591075,38.3,144
4,3,2017-01-05,37.583333,0.470084,38.3,144
...,...,...,...,...,...,...
1507592,32413,2021-12-27,34.013889,62.178514,214.0,144
1507593,32413,2021-12-28,93.618056,171.915575,531.0,144
1507594,32413,2021-12-29,67.291667,121.751603,410.0,144
1507595,32413,2021-12-30,69.659722,128.552521,440.0,144


## Dataset Meteo - sensori

### Attributi

- **IdSensore**
    - Foreign Key con vincolo di integrità referenziale a `IdSensore` di meteo_21
    - Testo normale
- **Tipologia**
    - Grandezza misurata
    - Testo normale
- **Unità DiMisura**
    - Unità di misura della grandezza
    - Testo normale
- IdStazione	
    - Numero identificativo della stazione (penso perchè ogni stazione può avere più sensori)
    - Testo normale
- NomeStazione	
    - Località della stazione (Spesso è 'Comune via' o 'Comune località)'
    - Testo normale
- Quota	
    - Altitudine
    - Numero
- Provincia
    - Sigla della provincia
    - Testo normale
- DataStart	
    - Data e ora
- DataStop	
    - Data e ora
- Storico	
    - Noto che assume valori N e S ma non capisco cosa sia
    - Testo normale
- UTM_Nord	
    - Coordinata UTM nord (le cordinate utm permettono di individuare univocamente un punto sulla cartina terrestre)
    - Testo normale
- UTM_Est	
    - Coordinata UTM est
    - Testo normale
- lng	
    - Longitudine (senza simbolo gradi)
    - Numero
- lat	
    - Latitudine (senza simbolo gradi)
    - Numero
- location	
    - (latitudine°, longitudine°)
    - Posizione

### Caratteristiche

Sono disponibili i dati delle grandezze:
- Livello Idrometrico (cm)
- Altezza neve (cm)
- Precipitazione (mm)
- Temperatura (°C)
- Umidità Relativa (%)
- Radiazione Globale (W/m2)
- Velocità e Direzione Vento (m/s e gradi).
- Velocità e direzione del vento (m/s e gradi N) raffica
NB: l’orario del dato è "ora solare" e si riferisce alle osservazioni ottenute fino all’orario indicato.

### Read_csv

In [7]:
sensori_meteo = pd.read_csv(f'{path}Stazioni_Meteorologiche.csv')
sensori_meteo.head()
#stazioni_meteo['Tipologia'].value_counts()

,IdSensore,Tipologia,Unità DiMisura,IdStazione,NomeStazione,Quota,Provincia,DataStart,DataStop,Storico,UTM_Nord,UTM_Est,lng,lat,location
0,22006,Umidità Relativa,%,1890,Tavernole sul Mella Monte Guglielmo,1790,BS,14/10/2019,NaN,N,5067376,591417,10.175407,45.753896,"(45.75389593743312, 10.175407094582125)"
1,22007,Direzione Vento,°,1890,Tavernole sul Mella Monte Guglielmo,1790,BS,14/10/2019,NaN,N,5067376,591417,10.175407,45.753896,"(45.75389593743312, 10.175407094582125)"
2,22003,Temperatura,°C,1890,Tavernole sul Mella Monte Guglielmo,1790,BS,14/10/2019,NaN,N,5067376,591417,10.175407,45.753896,"(45.75389593743312, 10.175407094582125)"
3,22009,Velocità Vento,m/s,1890,Tavernole sul Mella Monte Guglielmo,1790,BS,14/10/2019,NaN,N,5067376,591417,10.175407,45.753896,"(45.75389593743312, 10.175407094582125)"
4,17612,Altezza Neve,cm,1545,San Siro Alpe Rescascìa,1285,CO,07/11/2019,NaN,N,5102069,518430,9.238323,46.071947,"(46.071947417956004, 9.238323435918229)"


## Dataset Zonizzazione

In [8]:
zonizzazione = pd.read_csv(f'{path}zonizzazione/zonizzazione_ABCD.csv')
zonizzazione.head()

,Provincia,Codice Istat,Comune,Zona,Residenti 2008,Superficie (ha)
0,BG,16009,AMBIVERE,A,2341,327
1,BG,16013,ARZAGO D'ADDA,A,2836,944
2,BG,16018,BAGNATICA,A,4119,639
3,BG,16020,BARIANO,A,4396,714
4,BG,16021,BARZANO',A,5178,356


## Dataset pressione atmosferica

In [9]:
press_zama = pd.read_csv(f'{path}data_meteo/press/press_zama.csv', na_values = -9999)
press_zama.dtypes
press_zama.head()

,Data_ora,Valore
0,01/01/2016 23.59.00,1017.4
1,02/01/2016 23.59.00,1011.6
2,03/01/2016 23.59.00,1002.6
3,04/01/2016 23.59.00,993.0
4,05/01/2016 23.59.00,993.0


# Data Exploration e Data Cleansing (*superset*)

## Data Exploration

- verificare quali siano tutte le quantità uniche rilevate dai sensori qualità dell'aria
- quali siano i gruppi di inquinanti principali
- quali siano i limiti di assunzione umana (gionralieri e annuali)
    - Vedi [qui](https://www.arpalombardia.it/Pages/Aria/Inquinanti.aspx) per limiti e piccola descrizione
    - Vedi [qui](https://www.regione.lombardia.it/wps/portal/istituzionale/HP/DettaglioRedazionale/servizi-e-informazioni/cittadini/salute-e-prevenzione/Sicurezza-negli-ambienti-di-vita-e-di-lavoro/inquinamento-atmosferico/inquinamento-atmosferico/) per approfondimento

Determino il numero di sensori per ogni tipologia

- Si potrebbero considerare solo gli inquinanti **più importanti e con più stazioni**
- Ovvero **Biossido di Azoto | PM10 (SM2005) e PM2,5 e/o Particolato totale | Ozono troposferico | Bisossido di Zolfo**
- Facoltativi perchè non presenti nella valutazione della qualità dell'aria [qui](https://www.arpalombardia.it/Pages/Aria/Modellistica/Indice-qualit%C3%A0-aria.aspx): Monossido di carbonio e benzene

In [10]:
sensori_meteo['Tipologia'].value_counts()

Precipitazione         281
Temperatura            258
Umidità Relativa       199
Direzione Vento        144
Velocità Vento         144
Radiazione Globale     110
Livello Idrometrico     85
Altezza Neve            41
Name: Tipologia, dtype: int64

Determino il numero di zone in cui é suddiviso il territorio Lombardo

In [11]:
zonizzazione['Zona'].value_counts()

B         448
C         423
A         406
AGG MI    107
C D       102
AGG BG     37
AGG BS     20
D           2
Name: Zona, dtype: int64

## Data Cleansing

In [12]:
sensori_meteo.drop(sensori_meteo[~sensori_meteo['Tipologia'].isin(['Temperatura', 'Radiazione Globale'])].index, inplace = True)
sensori_meteo.reset_index(drop=True, inplace=True)
sensori_meteo

,IdSensore,Tipologia,Unità DiMisura,IdStazione,NomeStazione,Quota,Provincia,DataStart,DataStop,Storico,UTM_Nord,UTM_Est,lng,lat,location
0,22003,Temperatura,°C,1890,Tavernole sul Mella Monte Guglielmo,1790,BS,14/10/2019,NaN,N,5067376,591417,10.175407,45.753896,"(45.75389593743312, 10.175407094582125)"
1,22017,Radiazione Globale,W/m²,1890,Tavernole sul Mella Monte Guglielmo,1790,BS,14/10/2019,NaN,N,5067376,591417,10.175407,45.753896,"(45.75389593743312, 10.175407094582125)"
2,5865,Temperatura,°C,596,Osio Sotto v.per Levate,182,BG,04/04/2000,NaN,N,5052072,547691,9.611738,45.620556,"(45.62055567939559, 9.611737821812373)"
3,19293,Temperatura,°C,1514,Carenno centro sportivo,633,LC,08/05/2018,NaN,N,5072042,536210,9.465967,45.800989,"(45.80098941456421, 9.46596663838796)"
4,2190,Radiazione Globale,W/m²,114,Landriano Cascina Marianna,88,PV,29/09/2005,NaN,N,5018600,520938,9.267153,45.320594,"(45.32059361110084, 9.26715311477454)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
363,9827,Radiazione Globale,W/m²,835,Valdisotto Oga S. Colombano,2300,SO,01/10/1992,NaN,N,5145272,600293,10.305952,46.453563,"(46.45356305894081, 10.305951823970043)"
364,8008,Temperatura,°C,835,Valdisotto Oga S. Colombano,2300,SO,01/01/1992,NaN,N,5145272,600293,10.305952,46.453563,"(46.45356305894081, 10.305951823970043)"
365,12759,Temperatura,°C,1512,Pogliano Milanese,160,MI,25/11/2020,NaN,N,5042449,500406,9.005200,45.535577,"(45.535576719250145, 9.005199956067523)"
366,12755,Temperatura,°C,1510,Monza v.Monte Generoso,173,MB,25/11/2020,NaN,N,5048372,519174,9.245807,45.588626,"(45.58862552802921, 9.245807488805221)"


# Data preparation (group by e join su *superset*)

## JOIN meteo e sensori

In [13]:
meteo_sensori__id_day = pd.merge(meteo__id_day, sensori_meteo, how = 'inner', on = 'IdSensore')
meteo_sensori__id_day

,IdSensore,Data,Valore_mean,Valore_std,Valore_max,Valore_count,Tipologia,Unità DiMisura,IdStazione,NomeStazione,Quota,Provincia,DataStart,DataStop,Storico,UTM_Nord,UTM_Est,lng,lat,location
0,13,2017-01-01,-0.491667,2.179983,4.0,72,Temperatura,°C,6,Bormio Ponte sul Frodolfo,1250,SO,25/11/1987,NaN,N,5146642,605400,10.372743,46.465110,"(46.46511042549073, 10.372742774828684)"
1,13,2017-01-02,-1.538889,2.608035,4.1,72,Temperatura,°C,6,Bormio Ponte sul Frodolfo,1250,SO,25/11/1987,NaN,N,5146642,605400,10.372743,46.465110,"(46.46511042549073, 10.372742774828684)"
2,13,2017-01-03,-0.752778,1.479339,2.2,72,Temperatura,°C,6,Bormio Ponte sul Frodolfo,1250,SO,25/11/1987,NaN,N,5146642,605400,10.372743,46.465110,"(46.46511042549073, 10.372742774828684)"
3,13,2017-01-04,2.743056,2.741380,8.2,72,Temperatura,°C,6,Bormio Ponte sul Frodolfo,1250,SO,25/11/1987,NaN,N,5146642,605400,10.372743,46.465110,"(46.46511042549073, 10.372742774828684)"
4,13,2017-01-05,-3.972222,1.392760,-1.3,72,Temperatura,°C,6,Bormio Ponte sul Frodolfo,1250,SO,25/11/1987,NaN,N,5146642,605400,10.372743,46.465110,"(46.46511042549073, 10.372742774828684)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527961,12759,2021-12-27,4.034965,1.388091,5.6,143,Temperatura,°C,1512,Pogliano Milanese,160,MI,25/11/2020,NaN,N,5042449,500406,9.005200,45.535577,"(45.535576719250145, 9.005199956067523)"
527962,12759,2021-12-28,5.043750,1.134501,7.1,144,Temperatura,°C,1512,Pogliano Milanese,160,MI,25/11/2020,NaN,N,5042449,500406,9.005200,45.535577,"(45.535576719250145, 9.005199956067523)"
527963,12759,2021-12-29,4.531250,2.949706,10.8,144,Temperatura,°C,1512,Pogliano Milanese,160,MI,25/11/2020,NaN,N,5042449,500406,9.005200,45.535577,"(45.535576719250145, 9.005199956067523)"
527964,12759,2021-12-30,2.679167,0.754091,4.0,144,Temperatura,°C,1512,Pogliano Milanese,160,MI,25/11/2020,NaN,N,5042449,500406,9.005200,45.535577,"(45.535576719250145, 9.005199956067523)"


## Controllo significativitá medie e unitá di misura

### Medie ottenute da piú di un'osservazione

In [14]:
meteo_sensori__id_day.shape[0]

527966

In [15]:
meteo_sensori__id_day[meteo_sensori__id_day.Valore_count == 144].shape[0]

441723

In [16]:
meteo_sensori__id_day[meteo_sensori__id_day.Valore_count < 130].shape[0]

40407

#### Rimozione medie ottenute da meno di 130 osservazioni

In [17]:
meteo_sensori__id_day.drop(meteo_sensori__id_day[meteo_sensori__id_day.Valore_count < 130].index, inplace=True)
meteo_sensori__id_day.reset_index(drop=True, inplace=True)
meteo_sensori__id_day.head()

,IdSensore,Data,Valore_mean,Valore_std,Valore_max,Valore_count,Tipologia,Unità DiMisura,IdStazione,NomeStazione,Quota,Provincia,DataStart,DataStop,Storico,UTM_Nord,UTM_Est,lng,lat,location
0,13,2020-01-01,2.045390,1.888404,6.2,141,Temperatura,°C,6,Bormio Ponte sul Frodolfo,1250,SO,25/11/1987,NaN,N,5146642,605400,10.372743,46.46511,"(46.46511042549073, 10.372742774828684)"
1,13,2020-01-02,-0.215972,2.324773,5.1,144,Temperatura,°C,6,Bormio Ponte sul Frodolfo,1250,SO,25/11/1987,NaN,N,5146642,605400,10.372743,46.46511,"(46.46511042549073, 10.372742774828684)"
2,13,2020-01-03,1.945833,2.679131,7.1,144,Temperatura,°C,6,Bormio Ponte sul Frodolfo,1250,SO,25/11/1987,NaN,N,5146642,605400,10.372743,46.46511,"(46.46511042549073, 10.372742774828684)"
3,13,2020-01-04,5.172222,2.549796,9.7,144,Temperatura,°C,6,Bormio Ponte sul Frodolfo,1250,SO,25/11/1987,NaN,N,5146642,605400,10.372743,46.46511,"(46.46511042549073, 10.372742774828684)"
4,13,2020-01-05,4.391667,2.540126,8.9,144,Temperatura,°C,6,Bormio Ponte sul Frodolfo,1250,SO,25/11/1987,NaN,N,5146642,605400,10.372743,46.46511,"(46.46511042549073, 10.372742774828684)"


## Tengo solo MI

- Inner joint tra `meteo_sensori__id_day` e `zonizzazione` a dare `meteo_sensori_zone__id_day`
- Non perdo nessuna riga con l'inner joint perché sono presenti tutti i comuni nel dataset `zonizzazione`

In [18]:
#meteo_sensori_zone__id_day = pd.merge(meteo_sensori__id_day, zonizzazione['Zona'], left_on = meteo_sensori__id_day['Comune'].str.lower(), right_on = zonizzazione['Comune'].str.lower())
#meteo_sensori_zone__id_day.head()

In [19]:
#meteo_sensori_zone__id_day.drop(meteo_sensori_zone__id_day[meteo_sensori_zone__id_day['Zona'] != 'AGG MI'].index, inplace = True)
#meteo_sensori_zone__id_day.reset_index(drop = True, inplace = True)
#meteo_sensori_zone__id_day

In [20]:
meteo_sensori__id_day.drop(meteo_sensori__id_day[meteo_sensori__id_day['Provincia'] != 'MI'].index, inplace = True)
meteo_sensori__id_day.reset_index(drop=True, inplace=True)
meteo_sensori__id_day

,IdSensore,Data,Valore_mean,Valore_std,Valore_max,Valore_count,Tipologia,Unità DiMisura,IdStazione,NomeStazione,Quota,Provincia,DataStart,DataStop,Storico,UTM_Nord,UTM_Est,lng,lat,location
0,2001,2017-01-01,0.818881,4.269808,9.6,143,Temperatura,°C,100,Milano Lambrate,120,MI,29/06/2001,NaN,N,5038171,520120,9.257515,45.496780,"(45.49677982403948, 9.257515399681068)"
1,2001,2017-01-02,2.469444,2.840375,7.9,144,Temperatura,°C,100,Milano Lambrate,120,MI,29/06/2001,NaN,N,5038171,520120,9.257515,45.496780,"(45.49677982403948, 9.257515399681068)"
2,2001,2017-01-03,2.319444,3.701696,9.0,144,Temperatura,°C,100,Milano Lambrate,120,MI,29/06/2001,NaN,N,5038171,520120,9.257515,45.496780,"(45.49677982403948, 9.257515399681068)"
3,2001,2017-01-04,4.503472,4.753937,10.8,144,Temperatura,°C,100,Milano Lambrate,120,MI,29/06/2001,NaN,N,5038171,520120,9.257515,45.496780,"(45.49677982403948, 9.257515399681068)"
4,2001,2017-01-05,6.384722,1.438498,9.3,144,Temperatura,°C,100,Milano Lambrate,120,MI,29/06/2001,NaN,N,5038171,520120,9.257515,45.496780,"(45.49677982403948, 9.257515399681068)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39689,12759,2021-12-27,4.034965,1.388091,5.6,143,Temperatura,°C,1512,Pogliano Milanese,160,MI,25/11/2020,NaN,N,5042449,500406,9.005200,45.535577,"(45.535576719250145, 9.005199956067523)"
39690,12759,2021-12-28,5.043750,1.134501,7.1,144,Temperatura,°C,1512,Pogliano Milanese,160,MI,25/11/2020,NaN,N,5042449,500406,9.005200,45.535577,"(45.535576719250145, 9.005199956067523)"
39691,12759,2021-12-29,4.531250,2.949706,10.8,144,Temperatura,°C,1512,Pogliano Milanese,160,MI,25/11/2020,NaN,N,5042449,500406,9.005200,45.535577,"(45.535576719250145, 9.005199956067523)"
39692,12759,2021-12-30,2.679167,0.754091,4.0,144,Temperatura,°C,1512,Pogliano Milanese,160,MI,25/11/2020,NaN,N,5042449,500406,9.005200,45.535577,"(45.535576719250145, 9.005199956067523)"


## GROUP BY `meteo_sensori__day`

- Raggruppo per `Data` (giorno), `Zona`, `NomeTipoSensore`
- Calcolo grandezze aggregate
    - Calcolo il valore medio e la deviazione standard del `Valore_MEAN_id_day`, cioé il valore mediato di un sensore sull'intera giornata
    - Importante sottolineare che le funzioni `mean, count e std` di pandas NON tengono conto dei Na, quindi i valori che vediamo sono esatti
- Si perdono le colonne `Valore_count` e `Valore_STD_id_day` ma non sono importanti, sono piú importanti quelle che ricalcoliamo (chiamate `Valore_count_day_zona_tipo` e `Valore_STD_day_zona_tipo`)

In [21]:
meteo_sensori__day = meteo_sensori__id_day.groupby(['Data', 'Tipologia']).agg({'Valore_mean': ['mean', 'std'], 'Valore_max': ['mean', 'std']}).reset_index()

meteo_sensori__day.columns = ['Data', 'Tipologia', 'Valore_mean', 'Valore_std_mean', 'Valore_max','Valore_std_max']
meteo_sensori__day

temperatura = meteo_sensori__day[meteo_sensori__day['Tipologia'] == 'Temperatura'][['Data', 'Tipologia', 'Valore_max']]
radiazione_globale = meteo_sensori__day[meteo_sensori__day['Tipologia'] == 'Radiazione Globale'][['Data', 'Tipologia', 'Valore_mean','Valore_std_mean']]
temperatura.head()
radiazione_globale.head()

,Data,Tipologia,Valore_mean,Valore_std_mean
0,2017-01-01,Radiazione Globale,73.048141,20.409292
2,2017-01-02,Radiazione Globale,41.620833,13.619913
4,2017-01-03,Radiazione Globale,71.419246,3.751239
6,2017-01-04,Radiazione Globale,79.087401,4.263804
8,2017-01-05,Radiazione Globale,86.494072,4.794243


# Controlli (*superset*)

### Valori mancanti per ogni coppia `Zona` - `NomeTipoSensore`

#### Numero valori mancanti e valori NaN

I valori delle medie mancanti sono dovuti a:

- righe effettivamente mancanti
- righe con Valore_MEAN uguale a NaN

In [22]:
indice_date = pd.date_range(meteo_sensori__day.Data.min(), meteo_sensori__day.Data.max())
indice_date

DatetimeIndex(['2017-01-01', '2017-01-02', '2017-01-03', '2017-01-04',
               '2017-01-05', '2017-01-06', '2017-01-07', '2017-01-08',
               '2017-01-09', '2017-01-10',
               ...
               '2021-12-22', '2021-12-23', '2021-12-24', '2021-12-25',
               '2021-12-26', '2021-12-27', '2021-12-28', '2021-12-29',
               '2021-12-30', '2021-12-31'],
              dtype='datetime64[ns]', length=1826, freq='D')

#### Aggiunta valori NaN per giorni mancanti

Serve dopo quando si compie la media mobile (per mediare i giorni contigui e non le righe contigue)

In [23]:
temperatura = temperatura.set_index('Data').reindex(indice_date, fill_value=np.NaN).reset_index()
temperatura.columns = ['Data', 'Tipologia', 'Valore_max']
temperatura

,Data,Tipologia,Valore_max
0,2017-01-01,Temperatura,7.678571
1,2017-01-02,Temperatura,7.378571
2,2017-01-03,Temperatura,8.771429
3,2017-01-04,Temperatura,10.092857
4,2017-01-05,Temperatura,8.878571
...,...,...,...
1821,2021-12-27,Temperatura,6.206250
1822,2021-12-28,Temperatura,8.075000
1823,2021-12-29,Temperatura,9.518750
1824,2021-12-30,Temperatura,6.287500


In [24]:
radiazione_globale = radiazione_globale.set_index('Data').reindex(indice_date, fill_value=np.NaN).reset_index()
radiazione_globale.columns = ['Data', 'Tipologia', 'Valore_mean', 'Valore_std_mean']
radiazione_globale

,Data,Tipologia,Valore_mean,Valore_std_mean
0,2017-01-01,Radiazione Globale,73.048141,20.409292
1,2017-01-02,Radiazione Globale,41.620833,13.619913
2,2017-01-03,Radiazione Globale,71.419246,3.751239
3,2017-01-04,Radiazione Globale,79.087401,4.263804
4,2017-01-05,Radiazione Globale,86.494072,4.794243
...,...,...,...,...
1821,2021-12-27,Radiazione Globale,11.376786,1.899456
1822,2021-12-28,Radiazione Globale,23.749776,2.023651
1823,2021-12-29,Radiazione Globale,53.417758,11.747264
1824,2021-12-30,Radiazione Globale,31.981151,11.091993


Abbiamo ottenuto un numero di righe uguale a quelle teoriche con presenti dei NaN

# Medio/massimo negli anni

In [25]:
temperatura['month'] = temperatura.Data.dt.month
temperatura['day'] = temperatura.Data.dt.day
temperatura_year = temperatura.groupby(['month', 'day'], as_index = False).agg(Valore_max_mean =('Valore_max', 'mean'), Valore_max_std = ('Valore_max', 'std')).reset_index()
temperatura_year['year'] = 2020

temperatura_year['Data'] = pd.to_datetime(temperatura_year[['year', 'month', 'day']])

temperatura.drop(['month', 'day'], axis = 1, inplace = True)
temperatura_year.drop(['month', 'day', 'year'], axis = 1, inplace = True)
temperatura_year

,index,Valore_max_mean,Valore_max_std,Data
0,0,6.347744,3.055694,2020-01-01
1,1,9.534527,3.591281,2020-01-02
2,2,8.255363,0.623810,2020-01-03
3,3,9.407524,3.296207,2020-01-04
4,4,9.018952,2.798265,2020-01-05
...,...,...,...,...
361,361,6.514774,2.057278,2020-12-27
362,362,7.929286,3.650608,2020-12-28
363,363,7.588798,3.516738,2020-12-29
364,364,9.021690,4.668642,2020-12-30


In [26]:
radiazione_globale['month'] = radiazione_globale.Data.dt.month
radiazione_globale['day'] = radiazione_globale.Data.dt.day
radiazione_globale_year = radiazione_globale.groupby(['month', 'day'], as_index = False).agg(Valore_mean_max =('Valore_mean', 'max'), Valore_mean_mean =('Valore_mean', 'mean'), Valore_mean_std =('Valore_mean', 'std')).reset_index()
radiazione_globale_year['year'] = 2020

radiazione_globale_year['Data'] = pd.to_datetime(radiazione_globale_year[['year', 'month', 'day']])

radiazione_globale.drop(['month', 'day'], axis = 1, inplace = True)
radiazione_globale_year.drop(['month', 'day', 'year'], axis = 1, inplace = True)
radiazione_globale_year

,index,Valore_mean_max,Valore_mean_mean,Valore_mean_std,Data
0,0,75.617088,45.452980,30.360625,2020-01-01
1,1,86.009375,57.931202,32.406506,2020-01-02
2,2,71.419246,52.181673,15.663368,2020-01-03
3,3,79.087401,62.985493,25.733121,2020-01-04
4,4,86.494072,53.750202,35.743162,2020-01-05
...,...,...,...,...,...
361,361,60.957484,37.417460,27.707753,2020-12-27
362,362,72.034821,43.399349,22.198376,2020-12-28
363,363,74.004960,52.726936,22.426863,2020-12-29
364,364,68.918750,53.281244,14.121205,2020-12-30


In [27]:
anno = pd.date_range('2020-01-01', '2020-12-31')
valori = list(range(0, 366))
sin = [round(((m.sin(m.pi/366*(x + 11)))**2)*275 + 70, 4) for x in valori]

sinusoide = pd.DataFrame(sin, columns =['valore'], index = anno)
sinusoide.idxmax()

valore   2020-06-21
dtype: datetime64[ns]

## Pressione stazione Zama

In [28]:
press_zama.head()

,Data_ora,Valore
0,01/01/2016 23.59.00,1017.4
1,02/01/2016 23.59.00,1011.6
2,03/01/2016 23.59.00,1002.6
3,04/01/2016 23.59.00,993.0
4,05/01/2016 23.59.00,993.0


In [29]:
press_zama['Data'] = pd.to_datetime(press_zama['Data_ora'], format = '%d/%m/%Y %H.%M.%S')
press_zama['month'] = press_zama['Data'].dt.month
press_zama['day'] = press_zama['Data'].dt.day
press_zama_year = press_zama.groupby([press_zama['month'], press_zama['day']]).agg(press_mean =('Valore', 'mean'), press_std = ('Valore', 'std')).reset_index()
press_zama_year['year'] = 2020
press_zama_year['Data'] = pd.to_datetime(press_zama_year[['year', 'month', 'day']])

### calcolo percentili

In [50]:
print(np.nanpercentile(press_zama['Valore'], 5))
print(np.nanpercentile(press_zama['Valore'], 95))

993.3
1021.115


## Esportazione

In [30]:
sinusoide.to_csv(f'{path}/sinusoide.csv')

In [31]:
press_zama_year.to_csv(f'{path}/press_zama.csv')

In [32]:
temperatura_year.to_csv(f'{path}/temperatura.csv')
radiazione_globale_year.to_csv(f'{path}/radiazione_globale.csv')